In [2]:
import pandas as pd

In [4]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Classificação/Performance/churn_data.xlsx')
df.head()

,customerID,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,1,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,34,Yes,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,2,Yes,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,45,No,One year,No,Bank transfer,42.30,1840.75,No
4,9237-HQITU,2,Yes,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7042 entries, 0 to 7041
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7042 non-null   object 
 1   tenure            7042 non-null   int64  
 2   PhoneService      7042 non-null   object 
 3   Contract          7042 non-null   object 
 4   PaperlessBilling  7042 non-null   object 
 5   PaymentMethod     7042 non-null   object 
 6   MonthlyCharges    7042 non-null   float64
 7   TotalCharges      7042 non-null   object 
 8   Churn             7042 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 495.3+ KB


In [10]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [11]:
df = df.dropna()

In [21]:
cat_vars =['PhoneService','Contract', 'PaperlessBilling', 'PaymentMethod']
num_vars = ['tenure', 'MonthlyCharges', 'TotalCharges']

modeling_vars = cat_vars + num_vars

In [23]:
modeling_vars

['PhoneService',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'tenure',
 'MonthlyCharges',
 'TotalCharges']

In [25]:
x = df.filter(modeling_vars).copy()
y = df['Churn'].copy()
y = pd.DataFrame(y)

In [33]:
# ajustando as colunas com variáveis categoricas
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

le.fit(x.PhoneService)
x.PhoneService = le.transform(x.PhoneService)

le.fit(x.Contract)
x.Contract = le.transform(x.Contract)

le.fit(x.PaperlessBilling)
x.PaperlessBilling = le.transform(x.PaperlessBilling)

le.fit(x.PaymentMethod)
x.PaymentMethod = le.transform(x.PaymentMethod)

In [ ]:
# Y

le.fit(y.Churn)
y.Churn = le.transform(y.Churn)

In [ ]:
x = pd.get_dummies(x)
# usar se ver necessário

In [34]:
x.head()

,PhoneService,Contract,PaperlessBilling,PaymentMethod,tenure,MonthlyCharges,TotalCharges
0,0,0,1,2,1,29.85,29.85
1,1,1,0,3,34,56.95,1889.50
2,1,0,1,3,2,53.85,108.15
3,0,1,0,0,45,42.30,1840.75
4,1,0,1,2,2,70.70,151.65


In [36]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, stratify=y)

# Iniciando a melhora de performance

## GridSaerch

In [40]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

arvore = DecisionTreeClassifier()

parameters ={
            'max_depth':[1,2,3,4,5,6,7,8,9,10],
}

# Scoring é aquilo que queremos aumentar e melhorar
# Este parâmetro especifica o número de dobras (folds) para a validação cruzada. No exemplo, são usadas 5 dobras.
grid_search = GridSearchCV(arvore, parameters, scoring= 'roc_auc', cv=5, n_jobs=-1)


In [41]:
grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             scoring='roc_auc')

In [42]:
results = pd.DataFrame(grid_search.cv_results_)

In [45]:
results.sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,0.014145,0.000446,0.012608,0.000924,4,{'max_depth': 4},0.820900,0.816831,0.817485,0.832843,0.821944,0.822001,0.005760,1
4,0.015811,0.000468,0.013676,0.002874,5,{'max_depth': 5},0.827571,0.810666,0.816072,0.826468,0.825732,0.821302,0.006724,2
5,0.017783,0.001124,0.012083,0.001417,6,{'max_depth': 6},0.829359,0.797738,0.816262,0.814879,0.821995,0.816046,0.010481,3
2,0.013081,0.000748,0.012709,0.000796,3,{'max_depth': 3},0.811496,0.805850,0.812934,0.827466,0.813218,0.814193,0.007150,4
6,0.018422,0.000238,0.011658,0.000816,7,{'max_depth': 7},0.806625,0.776851,0.806352,0.805490,0.796437,0.798351,0.011396,5
7,0.021916,0.003232,0.011366,0.000676,8,{'max_depth': 8},0.795278,0.778171,0.778292,0.776782,0.790378,0.783780,0.007567,6
1,0.011241,0.001179,0.012414,0.001406,2,{'max_depth': 2},0.785204,0.761430,0.780380,0.798203,0.779745,0.780992,0.011820,7
8,0.021448,0.000465,0.012713,0.002987,9,{'max_depth': 9},0.786002,0.756049,0.749302,0.745481,0.777053,0.762777,0.015932,8
9,0.025047,0.003458,0.013781,0.004810,10,{'max_depth': 10},0.754363,0.732118,0.744663,0.744554,0.758395,0.746818,0.009130,9
0,0.009073,0.000858,0.012503,0.002805,1,{'max_depth': 1},0.727012,0.713869,0.716260,0.742985,0.732717,0.726568,0.010724,10


visto que o melhor parametro é o 4


## RandomForestClassifier

In [49]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [53]:
parameters = {
    'max_depth': [1,2,3,4,5,6,7,8,9,10],
    'n_estimators': [100,300,500,1000]
}

grid_search = GridSearchCV(estimator=rf, param_grid = parameters, scoring='roc_auc', cv=5, n_jobs=-1)

# Aqui temos novamnte o gridSearch em açõa para encontrar os melhores parametros a serem usados no modelo

In [55]:
grid_search.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [100, 300, 500, 1000]},
             scoring='roc_auc')

In [56]:
results = pd.DataFrame(grid_search.cv_results_)

In [58]:
results.sort_values(by='rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
23,6.690789,0.867104,0.251113,0.013251,6,1000,"{'max_depth': 6, 'n_estimators': 1000}",0.846893,0.829966,0.837657,0.848164,0.841478,0.840832,0.006619,1
27,6.814588,0.890941,0.264887,0.009790,7,1000,"{'max_depth': 7, 'n_estimators': 1000}",0.847954,0.829711,0.836904,0.848549,0.840936,0.840811,0.007062,2
21,2.305109,0.612804,0.115534,0.041613,6,300,"{'max_depth': 6, 'n_estimators': 300}",0.846047,0.829280,0.837432,0.848166,0.842363,0.840658,0.006756,3
20,0.636406,0.125212,0.034269,0.000927,6,100,"{'max_depth': 6, 'n_estimators': 100}",0.846480,0.830199,0.837592,0.846939,0.841486,0.840539,0.006209,4
26,3.743427,0.773254,0.149984,0.015874,7,500,"{'max_depth': 7, 'n_estimators': 500}",0.847387,0.829316,0.836288,0.848373,0.841117,0.840496,0.007108,5


### Vendo de fato quais seriam os melhores Hiperparametros a serem colocados

In [61]:
grid_search.best_estimator_.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 6,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1000,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

# RandomizedSearch

In [62]:
# Trabalha de uma forma aleatória e precisamos usar outro método
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_jobs=-1)

In [64]:
from sklearn.model_selection import RandomizedSearchCV

parameters ={
    'learning_rate': [0.001, 0.01],
    'num_leaves': [2,128],
    'min_child_samples': [1,100],
    'subsample':[0.05,1],
    'colsample_bytree': [0.1, 1.0]
}

random_search = RandomizedSearchCV(lgbm, parameters, scoring='roc_auc', cv=5, n_iter=5, n_jobs=-1)

random_search.fit(x_train,y_train)

[LightGBM] [Info] Number of positive: 1495, number of negative: 4129
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000585 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 594
[LightGBM] [Info] Number of data points in the train set: 5624, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265825 -> initscore=-1.015909
[LightGBM] [Info] Start training from score -1.015909


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


RandomizedSearchCV(cv=5, estimator=LGBMClassifier(n_jobs=-1), n_iter=5,
                   n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.1, 1.0],
                                        'learning_rate': [0.001, 0.01],
                                        'min_child_samples': [1, 100],
                                        'num_leaves': [2, 128],
                                        'subsample': [0.05, 1]},
                   scoring='roc_auc')

In [66]:
# Plotando os resultados
results = pd.DataFrame(random_search.cv_results_)
results.sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_num_leaves,param_min_child_samples,param_learning_rate,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.095262,0.952660,0.038681,0.013677,0.05,2,100,0.01,0.1,"{'subsample': 0.05, 'num_leaves': 2, 'min_chil...",0.813071,0.815074,0.813776,0.828474,0.823634,0.818806,0.006147,1
1,2.849231,1.873160,0.035768,0.005284,0.05,128,1,0.01,1.0,"{'subsample': 0.05, 'num_leaves': 128, 'min_ch...",0.819827,0.801133,0.802876,0.829391,0.821034,0.814852,0.011009,2
4,0.552038,0.552587,0.025485,0.009709,1,2,100,0.001,0.1,"{'subsample': 1, 'num_leaves': 2, 'min_child_s...",0.807188,0.813669,0.803078,0.822815,0.819120,0.813174,0.007298,3
3,2.417242,0.957373,0.026544,0.003181,0.05,128,1,0.001,0.1,"{'subsample': 0.05, 'num_leaves': 128, 'min_ch...",0.802544,0.803269,0.807081,0.807417,0.814172,0.806897,0.004132,4
2,1.023363,0.494657,0.048120,0.010396,0.05,2,1,0.01,1.0,"{'subsample': 0.05, 'num_leaves': 2, 'min_chil...",0.790846,0.789939,0.801190,0.807067,0.813650,0.800538,0.009178,5
